In [2]:
import numpy as np
import pandas as pd
from PIL import Image
import os
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split

In [3]:

img_path = "data/train" 
classes = ["burger", "masala_dosa", "momos", "pizza", "samosa"]


In [3]:
img_path_dict = {}
for class_name in classes:
    class_dir = os.path.join(img_path, class_name)
    for filename in os.listdir(class_dir):
        img_path_dict[filename] = class_name

In [4]:
df = pd.DataFrame(list(img_path_dict.items()), columns=["filename", "class"])

In [5]:
def load_images(metadata_path, img_dir, img_size=(30, 30)):
    metadata = pd.read_csv(metadata_path)
    images = []
    labels = []
    class_names = metadata['class'].unique()
    class_dict = {class_name: idx for idx, class_name in enumerate(class_names)}
    
    for index, row in metadata.iterrows():
        img_path = os.path.join(img_dir, row['class'], row['filename'])
        img = Image.open(img_path).resize(img_size)
        img = img.convert('RGB')
        arr = np.array(img)
        images.append(arr)
        labels.append(class_dict[row['class']])
    return np.array(images), np.array(labels)


In [ ]:

metadata_path = "data/Metafile1.csv"
x_train, y_train = load_images(metadata_path, img_path)


In [7]:
x_train = x_train / 255.0

In [8]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

In [9]:
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))

In [10]:
batch_size = 32
train_dataset = train_dataset.shuffle(buffer_size=len(x_train)).batch(batch_size)
val_dataset = val_dataset.batch(batch_size)

In [11]:
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(30, 30, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(len(classes), activation='softmax')
])

C:\Users\aarus\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [12]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
epochs = 15
history = model.fit(train_dataset, epochs=epochs, validation_data=val_dataset)

model.save("models/food_detection_model.h5")

val_loss, val_acc = model.evaluate(val_dataset)
print(f"Validation Accuracy: {val_acc * 100:.2f}%")

def predict_food(image_path, model, img_size=(30, 30)):
    img = Image.open(image_path).resize(img_size)
    img = img.convert('RGB')
    img_array = np.array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0) 
    prediction = model.predict(img_array)
    predicted_class = np.argmax(prediction, axis=1)
    return classes[predicted_class[0]]


Epoch 1/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 40ms/step - accuracy: 0.2227 - loss: 1.6089 - val_accuracy: 0.2764 - val_loss: 1.5894
Epoch 2/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - accuracy: 0.3519 - loss: 1.5543 - val_accuracy: 0.3869 - val_loss: 1.4546
Epoch 3/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - accuracy: 0.3653 - loss: 1.4776 - val_accuracy: 0.4070 - val_loss: 1.3532
Epoch 4/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.4879 - loss: 1.2934 - val_accuracy: 0.4070 - val_loss: 1.4066
Epoch 5/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.4737 - loss: 1.2703 - val_accuracy: 0.5427 - val_loss: 1.2014
Epoch 6/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.5812 - loss: 1.1045 - val_accuracy: 0.5578 - val_loss: 1.2195
Epoch 7/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.5864 - loss: 1.0431 - val_accuracy: 0.5930 - val_loss: 1.0325
Epoch 8/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.6372 - loss: 0.9837 - val_accuracy: 0.5628 - v

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.6242 - loss: 1.0054
Validation Accuracy: 63.32%


In [ ]:
model.save("models/my_model.keras")
model = tf.keras.models.load_model('models/my_model.keras')


In [4]:
from flask import Flask, request, jsonify, render_template_string
from pyngrok import ngrok
import io

In [ ]:

model = tf.keras.models.load_model("models/my_model.keras")

def preprocess_image(image):
    image = image.resize((30, 30))
    image = image.convert('RGB')
    image = np.array(image) / 255.0
    image = np.expand_dims(image, axis=0)
    return image

app = Flask(__name__)

@app.route('/')
def upload_form():
    return render_template_string("""
  <!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Food Detection System</title>
    <style>
         body {
            font-family: 'Arial', sans-serif;
            margin: 0;
            padding: 0;
            background-color: #f4f4f9;
            background-image:url("static/ep1.jpg");
            color: #333;
            
            background-size: 1600px 800px;
        }
        .navbar {
            background-color: #333;
            overflow: hidden;
            padding: 10px 20px;
            box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1);
        }
        .navbar a {
            float: left;
            display: block;
            color: #f2f2f2;
            text-align: center;
            padding: 14px 16px;
            text-decoration: none;
            font-size: 18px;
        }
        .navbar a:hover {
            background-color: #575757;
            border-radius: 5px;
        }
        .container {
            text-align: center;
            padding: 20px;
            max-width: 800px;
            margin: 0 auto;
        }
        h1 {
            font-size: 3em;
            color: #ffd700;
            margin-bottom: 20px;
            text-shadow: 2px 2px 4px rgba(0, 0, 0, 1);
        }
        .dropbox {
            border: 2px dashed #888;
            padding: 30px;
            border-radius: 10px;
            background-color: #fafafa;
            cursor: pointer;
            margin-top: 20px;
            transition: background-color 0.3s, border-color 0.3s;
        }
        .dropbox.dragover {
            border-color: #555;
            background-color: #e8e8e8;
        }
        #uploaded-image {
            max-width: 100%;
            margin-top: 20px;
            border-radius: 10px;
            box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1);
        }
        .result {
            margin-top: 20px;
            font-size: 1.2em;
            color: #444;
            padding: 10px;
            background-color: #e0f7fa;
            border-radius: 10px;
        }
        .about-cnn {
            margin-top: 5px;
            text-align: left;
            background-color:rgba(red, green, blue, 0);
            padding: 20px;
            border-radius: 10px;
            box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1);
        }
        .about-cnn h2 {
            color: #f3f30f;
            margin-bottom: 10px;
            text-shadow: 2px 4px 6px rgba(0, 0, 0, 0.5);
        }
        .about-cnn p {
            font-size: 1.1em;
            line-height: 1.6;
            color: #ebeb4c;
            text-shadow: 2px 4px 6px rgba(0, 0, 0, 1);
        }
        canvas {
            position: fixed;
            top: 0;
            left: 0;
            z-index: -1;
            width: 100%;
            height: 100%;
        }
    </style>
</head>
<body>
    <nav class="navbar">
        <a href="#">Home</a>
        <a href="#about">About CNN</a>
        <a href="#upload">Upload Image</a>
    </nav>
    <canvas id="background-canvas"></canvas>
    <div class="container">
        <h1>Food Detection System using CNN</h1>
        <div id="upload">
            <div id="dropbox" class="dropbox">Drag and drop an image here or click to select</div>
            <input type="file" id="file-input" accept="image/*" style="display: none;">
            <img id="uploaded-image" src="" alt="Uploaded Image" hidden>
            <div id="result" class="result" hidden></div>
        </div>
        <div class="about-cnn" id="about">
            <h2>About Convolutional Neural Networks (CNN)</h2>
            <p>
                Convolutional Neural Networks (CNNs) are a class of deep learning models specifically designed for image processing tasks. They use convolutional layers to automatically and adaptively learn spatial hierarchies of features from input images. CNNs are widely used in applications like image classification, object detection, and facial recognition.
            </p>
            <p>
                In this Food Detection System, a CNN model is trained to classify food images into different categories. The model processes the input image, extracts features, and predicts the most likely food class.
            </p>
        </div>
    </div>

    <script>
        const dropbox = document.getElementById('dropbox');
        const fileInput = document.getElementById('file-input');
        const uploadedImage = document.getElementById('uploaded-image');
        const resultDiv = document.getElementById('result');

        dropbox.addEventListener('click', () => fileInput.click());

        dropbox.addEventListener('dragover', (event) => {
            event.preventDefault();
            dropbox.classList.add('dragover');
        });

        dropbox.addEventListener('dragleave', () => dropbox.classList.remove('dragover'));

        dropbox.addEventListener('drop', (event) => {
            event.preventDefault();
            dropbox.classList.remove('dragover');
            const file = event.dataTransfer.files[0];
            if (file) handleFile(file);
        });

        fileInput.addEventListener('change', () => {
            const file = fileInput.files[0];
            if (file) handleFile(file);
        });

        function handleFile(file) {
            if (!file.type.startsWith('image/')) {
                alert('Please upload a valid image file.');
                return;
            }

            const reader = new FileReader();
            reader.onload = () => {
                uploadedImage.src = reader.result;
                uploadedImage.hidden = false;
                sendImageToServer(file);
            };
            reader.readAsDataURL(file);
        }

        function sendImageToServer(file) {
            const formData = new FormData();
            formData.append('image', file);

            fetch(`${window.location.origin}/predict`, {
                method: 'POST',
                body: formData
            })
                .then(response => response.json())
                .then(data => {
                    resultDiv.textContent = `The Predicted Class is ${data.class}`;
                    resultDiv.hidden = false;
                })
                .catch(error => {
                    resultDiv.textContent = 'Error: Unable to make prediction.';
                    resultDiv.hidden = false;
                    console.error(error);
                });
        }

    </script>
</body>
</html>
    
                
        
    """)

@app.route('/predict', methods=['POST'])
def predict():
    file = request.files['image']
    image = Image.open(file.stream)
    processed_image = preprocess_image(image)
    prediction = model.predict(processed_image)
    predicted_class = classes[np.argmax(prediction)]
    return jsonify({'class': predicted_class})

if __name__ == '__main__':
    ngrok_token = os.getenv('NGROK_AUTH_TOKEN')
    if ngrok_token:
        ngrok.set_auth_token(ngrok_token)
        public_url = ngrok.connect(5000)
        print(f"Public URL: {public_url}")
    else:
        print("Warning: NGROK_AUTH_TOKEN not set. Running without ngrok.")
    app.run(host='0.0.0.0', port=5000)


c:\Users\aarus\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 22 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Public URL: NgrokTunnel: "https://22994da991b7.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.1.3:5000
Press CTRL+C to quit
127.0.0.1 - - [04/Jan/2026 18:30:15] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [04/Jan/2026 18:30:15] "GET /static/ep1.jpg HTTP/1.1" 404 -
127.0.0.1 - - [04/Jan/2026 18:30:16] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [04/Jan/2026 18:32:43] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [04/Jan/2026 18:32:43] "GET /static/ep1.jpg HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 569ms/step


127.0.0.1 - - [04/Jan/2026 18:32:57] "POST /predict HTTP/1.1" 200 -
